# Loading and preprocessing datasets

## Imports

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy
import anndata as ad


In [ ]:
import sys
sys.path.insert(0, "../utils")
from utils import preprocess

## NeuroIPS

In [ ]:
X = sc.read_h5ad('GSE194122_neurips2021/raw/GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad',)

In [ ]:
X = X[:,X.var.feature_types == 'ATAC']

In [ ]:
mapper = lambda x: '{}:{}-{}'.format(*x.split('-'))

In [ ]:
adata.var_names = adata.var_names.map(mapper)

In [ ]:
discrete = ['cell_type', 'batch', 'Site', 'DonorID',  'DonorBloodType',
       'DonorRace', 'Ethnicity', 'DonorGender', 'DonorSmoker']
cont = ['DonorAge', 'DonorBMI']

In [ ]:
adata = preprocess(adata, discrete, cont, frac = 0.1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=5)

In [ ]:
adata.write_h5ad("Processed_benchmark_datasets/NeuroIPS.h5ad")

## sciATAC-seq3

In [ ]:
path = 'sciATAC3/RDS/'
path_out = 'sciATAC3/anndata/'

In [ ]:
datasets = ['GSM4508931_eye_filtered'      ,
'GSM4508935_liver_filtered'     ,
'GSM4508939_placenta_filtered',
'GSM4508928_adrenal_filtered'  ,
'GSM4508932_heart_filtered'    ,  
'GSM4508936_lung_filtered'     , 
'GSM4508940_spleen_filtered',
'GSM4508929_cerebellum_filtered'  ,
'GSM4508933_intestine_filtered'  ,
'GSM4508937_muscle_filtered'   , 
'GSM4508941_stomach_filtered',
'GSM4508930_cerebrum_filtered'  ,  
'GSM4508934_kidney_filtered'   ,  
'GSM4508938_pancreas_filtered'  ,
'GSM4508942_thymus_filtered']

In [ ]:
for dataset in datasets:
    X = sc.read_mtx(f'{path}{dataset}.matrix.mtx', )
    X = X.X.T
    X = sc.AnnData(X,
               obs = pd.read_csv(f'{path}{dataset}.barcodes.tsv',sep = ' '),
               var = pd.read_csv(f'{path}{dataset}.features.tsv',sep = ' ', index_col = 1))
    X.X = scipy.sparse.csr_matrix(X.X)
    X.write_h5ad(f'{path_out}{dataset}.h5ad')

In [ ]:
datasets = [f'{path_out}{dataset}.h5ad' for dataset in datasets]

In [ ]:
import anndata as ad
ad.experimental.concat_on_disk(
    datasets,
    f"{path_out}merged.h5ad",
    label="dataset",
)

In [ ]:
adata = sc.read_h5ad(f"{path_out}merged.h5ad")

In [ ]:
discrete = ['donor_id', 'sex', 'batch', 'tissue', 'cell_type']
cont = ['day_of_pregnancy']

In [ ]:
adata = preprocess(adata, discrete, cont, frac = 0.1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=5)

In [ ]:
mapper = lambda x: '{}:{}-{}'.format(*x.split('-'))

In [ ]:
adata.var_names = adata.var_names.map(mapper)

In [ ]:
adata.write_h5ad("Processed_benchmark_datasets/sciATAC3.h5ad")

## GSE162170

In [ ]:
path = 'GSE162170/raw/'
path_out = 'GSE162170/anndata/'

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import anndata

file_path = f'{path}GSE162170_atac_counts.tsv'

chunk_size = 1000 
sparse_data = []
rows = []
cols = []
row_counter = 0

for chunk in pd.read_csv(file_path, sep='\t', header=None, chunksize=chunk_size):
    non_zero_entries = np.where(chunk.values != 0)
    sparse_data.extend(chunk.values[non_zero_entries].tolist())
    rows.extend((non_zero_entries[0] + row_counter).tolist())
    cols.extend(non_zero_entries[1].tolist())
    
    row_counter += chunk.shape[0]

sparse_matrix = csr_matrix((sparse_data, (rows, cols)), shape=(row_counter, chunk.shape[1]))



In [ ]:
obs = pd.read_csv(f'{path}GSE162170_atac_cell_metadata.txt',sep = '\t', index_col = 0)
var = pd.read_csv(f'{path}GSE162170_atac_consensus_peaks.bed',sep = '\t', index_col =None, header = None)

In [ ]:
var.index = [f"{x[0]}:{x[1]}-{x[2]}" for y, x in var.iterrows()]

In [ ]:
adata = anndata.AnnData(sparse_matrix.T, 
               var = var, obs = obs)

In [ ]:
adata.var = adata.var[[0, 1, 2]]
adata.var.columns = ['chr', 'start', 'end']

In [ ]:
adata.obs['Age_int'] = [float(x[3:]) for x in adata.obs.Age]

In [ ]:
adata = preprocess(adata, discrete, cont, frac = 0.1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=5)

In [ ]:
adata.write_h5ad("Processed_benchmark_datasets/Gestation_cerebral_cortex.h5ad")

## GSE151302

In [ ]:
path = 'GSE151302_kidney/raw/'
path_out = 'GSE151302_kidney/anndata/'

In [ ]:
datasets = ['GSM4572187_Control1_fragments.tsv.gz',
            'GSM4572188_Control2_fragments.tsv.gz',
            'GSM4572189_Control3_fragments.tsv.gz',
            'GSM4572190_Control4_fragments.tsv.gz',
            'GSM4572191_Control5_fragments.tsv.gz',  
           ]


In [ ]:
datasets = [f'{path}{dataset}' for dataset in datasets]

In [ ]:
metadata = {'Control_1':{'age': 54,
        'gender': 'Male',
        'race': 'NHW',
        'egfr': 58,
        'global glomerulosclerosis': '< 10%',
        'ifta': '1-10%',
        'tissue': 'kidney'},
 'Control_2':{'age': 62,
        'gender': 'Male',
        'race': 'HIS',
        'egfr': 61,
        'global glomerulosclerosis': '< 10%',
        'ifta': '1-10%',
        'tissue': 'kidney'},
 'Control_3':{'age': 61,
        'gender': 'Female',
        'race': 'NHW',
        'egfr': 69,
        'global glomerulosclerosis': '< 10%',
        'ifta': '1-10%',
        'tissue': 'kidney'},
 'Control_4':{'age': 50,
        'gender': 'Male',
        'race': 'NHW',
        'egfr': 78,
        'global glomerulosclerosis': '< 10%',
        'ifta': '1-10%',
        'tissue': 'kidney'},
 'Control_5':{'age': 52,
        'gender': 'Female',
        'race': 'NHW',
        'egfr': 98,
        'global glomerulosclerosis': '< 10%',
        'ifta': '1-10%',
        'tissue': 'kidney'},
}
 

In [ ]:
barcodes = pd.read_csv(f"{path}atac_barcodes.csv")

In [ ]:
samples = sorted(list(barcodes['sample'].unique()))

In [ ]:
data = snapatac2.pp.import_data(
    datasets,
    chrom_sizes=snapatac2.genome.hg38,
    sorted_by_barcode=False,
)

In [ ]:
for idx, d in enumerate(data):
     d.obs_names = [f"{x.split('-')[0]}-{idx +1}" for x in d.obs_names]

In [ ]:
for idx, i in enumerate(samples):
    match = data[idx].obs_names.isin(list(barcodes[barcodes['sample'] == i].barcode))
    data[idx] = data[idx][match].copy()

In [ ]:
for idx, d in enumerate(data):
    snapatac2.pp.make_peak_matrix(d, inplace=True, backend='hdf5', peak_file=f"{path}atac_peaks.bed",)

In [ ]:
for d in data:
    d.obsm = None


In [ ]:
for idx, d in enumerate(data):
    d.write_h5ad(f"{path_out}{samples[idx]}.h5ad")
data = []
for idx in range(5):
    data.append(sc.read_h5ad(f"{path_out}{samples[idx]}.h5ad"))

In [ ]:
adata = ad.concat(data)

In [ ]:
barcodes.index = barcodes.barcode

In [ ]:
barcodes = barcodes.loc[adata.obs_names]

In [ ]:
ls = ['barcode', 'sample', 'celltype']

In [ ]:
adata.obs[ls] = barcodes[ls]

In [ ]:
df = {idx:metadata[i] for idx, i in zip(adata.obs.index, adata.obs['sample'])}
df = pd.DataFrame(df)

In [ ]:
df = df.T

In [ ]:
ls = df.columns

In [ ]:
adata.obs[ls] = df[ls]

In [ ]:
adata.obs.age = adata.obs.age.map(int)
adata.obs.egfr = adata.obs.egfr.map(int)
adata.obs.age = adata.obs.age.map(int)

In [ ]:
discrete = ['sample', 'celltype', 'age', 'gender', 'race',]
cont = ['egfr']

In [ ]:
adata = preprocess(adata, discrete, cont, frac = 0.1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=5)

In [ ]:
adata.write_h5ad("Processed_benchmark_datasets/Kidney.h5ad")

## Pancreas

In [4]:
path = '/work/Perturbation/Data/ATAC_seq_datasets/pancreas'

In [5]:
adata = sc.read_h5ad(f'{path}/anndata/pancreas.h5ad')

In [6]:
ct_mapper = {'0': 'Alpha',
 '1': 'Beta',
 '2': 'Beta',
 '3': 'Beta',
 '4': 'Alpha',
 '5': 'Beta',
 '6': 'Delta',
 '7': 'Acinar',
 '8': 'Alpha',
 '9': 'Ductal',
 '10': 'Gamma',
 '11': 'Stellate',
 '12': 'Immune',
 '13': 'EC'}

In [10]:
anno = 'GSE169453_cell_annotation.csv'
path = '/work/Perturbation/Data/ATAC_seq_datasets/pancreas'
anno = pd.read_csv(f'{path}/{anno}')

In [11]:
adata.obs['leiden'] = anno.set_index('index')['leiden'][adata.obs.index].map(str)
adata.obs['cell_type'] = adata.obs.leiden.map(ct_mapper)

In [12]:
adata.obs['Purity'] = adata.obs['Purity'].map(lambda x:float(x.replace(',', '.')))

In [13]:
discrete = ['donor', 'cell_type', 'Diabetes status', 'Human islet resource center', 'Sex', 'Ethnicity', ' 10x multiome assay']
cont = ['Islet index', 'Age', 'BMI', 'HbA1c', 'Purity']


In [ ]:
adata = preprocess(adata, discrete, cont, frac = 0.1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=5)

In [ ]:
def rename_var(var_name):
    if ':' in var_name:
        return var_name
    else:
        return f"{var_name}:10000-15000"

In [ ]:
adata.var_names = adata.var_names.map(rename_var)

In [ ]:
adata.write_h5ad("Processed_benchmark_datasets/Pancreas_ATAC.h5ad")